In [1]:
import pandas as pd
from itertools import chain
import plotly.express as px

# Pipeline imports
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer, StandardScaler, MinMaxScaler, PCA
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import count, when, isnan, col, log1p, log, create_map, lit
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("BD").getOrCreate()

In [2]:
def get_numerical(data):
    """
    Returns a list of column names that have numerical data types (int or double).
    """
    return [t[0] for t in data.dtypes if t[1] == 'int' or t[1] == 'double']


def plot_hist(data, col):
    fig = px.histogram(data.toPandas(), x=col)
    fig.show()


def clean(data):
    """
    Cleans the given data by filling missing values, dropping rows with missing values,
    and mapping categorical variables to numerical values.
    """
    data = data.fillna(0, subset=["tempo"])
    data = data.dropna()

    keys = {'A': 0, 'A#': 1, 'B': 2, 'C': 3, 'C#': 4, 'D': 5, 'D#': 6, 'E': 7, 'F': 8, 'F#': 9, 'G': 10, 'G#': 11}
    key_mapping = create_map([lit(x) for x in chain(*keys.items())])
    data = data.withColumn("key", key_mapping[col("key")])

    modes = { 'Minor': 0, 'Major': 1 }
    mode_mapping = create_map([lit(x) for x in chain(*modes.items())])
    data = data.withColumn("mode", mode_mapping[col("mode")])
    
    return data


def norm_data(data, col, mean=None, std=None, maxv=None, minv=None):
    """
    Normalize the data in the specified column within the given DataFrame.

    Args:
        data (DataFrame): The input DataFrame.
        col (str): The name of the column to normalize.
        mean (float, optional): The mean value to use for normalization. If not provided, it will be calculated from the data.
        std (float, optional): The standard deviation value to use for normalization. If not provided, it will be calculated from the data.
        maxv (float, optional): The maximum value to use for normalization. If not provided, it will be calculated from the data.
        minv (float, optional): The minimum value to use for normalization. If not provided, it will be calculated from the data.

    Returns:
        Tuple[DataFrame, float, float, float, float]: A tuple containing the normalized DataFrame, mean, standard deviation, maximum value, and minimum value.

    """
    
    if mean is None: mean = data.agg({col: "mean"}).collect()[0][0]
    if std is None: std = data.agg({col: "stddev"}).collect()[0][0]

    # outlier
    data = data.withColumn(f"{col}_outlier", when((data[col] < mean - 3 * std) | (data[col] > mean + 3 * std), 1).otherwise(0))
    data = data.withColumn(col, when(data[col] < mean - 3 * std, mean - 3 * std).otherwise(data[col]))
    data = data.withColumn(col, when(data[col] > mean + 3 * std, mean + 3 * std).otherwise(data[col]))

    # normalize [0,1] range
    if maxv is None: maxv = data.agg({col: "max"}).collect()[0][0]
    if minv is None: minv = data.agg({col: "min"}).collect()[0][0]
    data = data.withColumn(f"{col}", (data[col] - minv) / (maxv - minv))

    # delete outlier column
    data = data.drop(f"{col}_outlier")

    return data, mean, std, maxv, minv

In [6]:
label = ["music_genre"]
seed = 1235
data = spark.read.csv("../data/training.csv", header=True, inferSchema=True, nullValue="", sep=";")
data = clean(data)
dev, _ = data.randomSplit([0.9, 0.1], seed=seed)


# dev processing
col_params = {}
for col_name in get_numerical(dev):
    dev, mean, std, maxv, minv = norm_data(dev, col_name)
    col_params[col_name] = (mean, std, maxv, minv)
    # plot_hist(dev, col_name)



In [10]:
label_col = "music_genre_index"
indexer = StringIndexer(inputCols=label, outputCols=[label_col])
assembler = VectorAssembler(inputCols=get_numerical(dev), outputCol="features")
estimator = KMeans(k=10, featuresCol="features")
pipeline = Pipeline(stages=[indexer, assembler, estimator])

# params = (ParamGridBuilder()
#     .build())
# evaluator = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol="prediction", metricName="f1")
# cross = CrossValidator(estimator=pipeline, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)

# build and train the pipeline
model = pipeline.fit(dev)

In [27]:
_, test = data.randomSplit([0.9, 0.1], seed=seed)

# test processing
for col_name in get_numerical(test):
    test, _, _, _, _ = norm_data(test, col_name, *col_params[col_name])
    # plot_hist(test, col_name)


# evaluate the model
centroids = model.stages[-1].clusterCenters()
print(centroids)
# use the centroids to predict the cluster for each data point
predictions = model.transform(test)
predictions = predictions.withColumn("prediction", predictions["prediction"].cast("double"))
predictions = predictions.withColumn(label_col, predictions[label_col].cast("double"))
# convert features to a vector

from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.types import ArrayType, DoubleType

# Define a UDF to convert Vector to Array
vector_to_array = udf(lambda v: v.toArray().tolist(), ArrayType(DoubleType()))
predictions = predictions.withColumn("features_arr", vector_to_array(predictions["features"]))

# display(predictions.limit(10).toPandas())
# calculate the distance between each data point and the corresponding cluster centroid
for i in range(len(centroids)):
    predictions = predictions.withColumn(f"distance_{i}", sum((predictions["features_arr"][j] - centroids[i][j]) ** 2 for j in range(len(centroids[i]))))

display(predictions.limit(10).toPandas())
# # calculate whether the prediction is correct
# predictions = predictions.withColumn("is_correct", when(test("prediction") == test(label_col), 1).otherwise(0))

# # calculate the accuracy
# accuracy = predictions.agg({"is_correct": "mean"}).collect()[0][0]
# print(f"accuracy: {accuracy}")

[array([0.40411414, 0.82223417, 0.39526801, 0.37891885, 0.22846783,
       0.04075774, 0.47864611, 0.25800681, 0.40910916, 1.        ,
       0.06839788, 0.4405602 , 0.2517767 ]), array([0.33816082, 0.85739039, 0.33081663, 0.42228986, 0.18024557,
       0.84781475, 0.51000855, 0.19815319, 0.25083758, 1.        ,
       0.0663467 , 0.4339261 , 0.23744101]), array([0.49773122, 0.58010529, 0.60715336, 0.31379181, 0.49343069,
       0.04521976, 0.50977178, 0.24324815, 0.57759172, 1.        ,
       0.14571375, 0.50910636, 0.59285126]), array([0.52181313, 0.16663399, 0.57455952, 0.36168117, 0.70855574,
       0.02495552, 0.47865124, 0.27199471, 0.67283745, 0.        ,
       0.1191819 , 0.49841731, 0.49853797]), array([0.59470727, 0.20046555, 0.69390008, 0.32995955, 0.66623092,
       0.01389964, 0.4302926 , 0.30764568, 0.66348298, 0.        ,
       0.69495024, 0.49052978, 0.51418818]), array([0.39819476, 0.18378586, 0.58037063, 0.40598522, 0.66176535,
       0.76204844, 0.44587368, 0.2628

In [ ]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.types import ArrayType, DoubleType

# Define a UDF to convert Vector to Array
vector_to_array = udf(lambda v: v.toArray().tolist(), ArrayType(DoubleType()))
predictions = predictions.withColumn("features_arr", vector_to_array(predictions["features"]))

# Now you can access the elements of features
for i in range(len(centroids)):
    predictions = predictions.withColumn(f"distance_{i}", sum((predictions["features_arr"][j] - centroids[i][j]) ** 2 for j in range(len(centroids[i]))))

In [6]:
_, test = data.randomSplit([0.9, 0.1], seed=seed)
print("test count: ", test.count())

# test processing
for col_name in get_numerical(test):
    test, _, _, _, _ = norm_data(test, col_name, *col_params[col_name])
    # plot_hist(test, col_name)

print("test count: ", test.count())

# evaluate the model
predictions = model.transform(test)
f1 = evaluator.evaluate(predictions)
print(f"f1: {f1}")

test count:  3029
test count:  3029
f1: 0.5869630239773582


In [28]:

for col in numerical_features:
    mean = data.agg({col: "mean"}).collect()[0][0]
    std = data.agg({col: "stddev"}).collect()[0][0]
    # print(f"Mean: {mean}, Std: {std}")

    # use mean a std to calculate outliers
    data = data.withColumn(f"{col}_outlier", when((data[col] < mean - 3 * std) | (data[col] > mean + 3 * std), 1).otherwise(0))
    outliers = data.filter(f"{col}_outlier = 1")
    print(f"{col}: {outliers.count()}")

    # correct outliers
    # data = data.withColumn(col, when(data[col] < mean - 3 * std, mean - 3 * std).otherwise(data[col]))
    # data = data.withColumn(col, when(data[col] > mean + 3 * std, mean + 3 * std).otherwise(data[col]))

    # remove outliers
    data = data.filter(f"{col}_outlier = 0")
    
    # set outliers to mean
    # data = data.withColumn(col, when(data[f"{col}_outlier"] == 1, mean).otherwise(data[col]))

    # center in 0
    # data = data.withColumn(f"{col}_norm", (data[col] - mean) / std)

    # between 1 and 0
    max = data.agg({col: "max"}).collect()[0][0]
    min = data.agg({col: "min"}).collect()[0][0]
    data = data.withColumn(f"{col}_norm", (data[col] - min) / (max - min))
    
    




popularity: 8
acousticness: 0
danceability: 0
duration_ms: 284
energy: 0
instrumentalness: 0
liveness: 834
loudness: 679
speechiness: 631
tempo: 0
valence: 0
key_num: 0
mode_num: 0


KeyboardInterrupt: 

In [45]:
data.agg({"valence": "mean"}).collect()[0][0]

0.4554741183921099